### Banco maquina local

Nesse notebook, ao invez de utilizar uma VPS da AWS, esse banco está instalado em uma maquina física

In [1]:
#importações

import pandas as pd
import pyodbc
from datetime import datetime
from pegar_historico import pegar_historico
from dotenv import load_dotenv
import os
from datetime import date

In [15]:


#Parametros para estabecer a conexão com o banco
load_dotenv() 

connection_string = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={os.getenv('DB_HOST')},{os.getenv('DB_PORT')};
    DATABASE={os.getenv('DB_NAME')};
    UID={os.getenv('DB_USER')};
    PWD={os.getenv("DB_PASSWORD")};
    Encrypt=yes;
    TrustServerCertificate=yes;
"""



conn = pyodbc.connect(connection_string)

#Para checar a conexao, estou puxando dados de um banco e tabela que já existem nesse servidor
query = "SELECT * FROM Dim_Categoria;"
df = pd.read_sql(query, conn)

# Mostrando o DataFrame
print(df)

conn.close()

      Cod_Categoria    Desc_Categoria
0   Ãguas Minerais   Ãguas Minerais
1    Suco de Frutas    Suco de Frutas
2   Ãguas Minerais   Ãguas Minerais
3  ÃguasÂ Minerais  ÃguasÂ Minerais
4              Mate              Mate
5    Suco de Frutas    Suco de Frutas


C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_20832\1195107787.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Todavia é interessante criar um banco e tabela nova utilizando essa conexao 

In [20]:
# Conectando ao servidor, usando o banco 'master' (obrigatório para criar novos databases)

# Abrir conexão
conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()

# Nome do novo banco de dados
new_db_name = 'bitcoin_stats'

# Comando para criar o banco
create_db_query = f"CREATE DATABASE {new_db_name};"

try:
    cursor.execute(create_db_query)
    print(f"Banco de dados '{new_db_name}' criado com sucesso!")
except Exception as e:
    print("Erro ao criar banco de dados:", e)

# Fechar conexão
cursor.close()
conn.close()


Banco de dados 'bitcoin_stats' criado com sucesso!


## Criando uma tabela no novo banco

In [65]:
#Primeiro, checo quais tabelas já existem no banco
def checa_tabelas_existentes():
    conn = pyodbc.connect(connection_string, autocommit=True)
    
    query = """
    SELECT *
    FROM INFORMATION_SCHEMA.TABLES;
    """
    
    df = pd.read_sql(query, conn)
    
    conn.close()
    
    return df

checa_tabelas_existentes()

C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_20832\4279729226.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,master,dbo,spt_fallback_db,BASE TABLE
1,master,dbo,spt_fallback_dev,BASE TABLE
2,master,dbo,spt_fallback_usg,BASE TABLE
3,master,dbo,spt_values,VIEW
4,master,dbo,spt_monitor,BASE TABLE
5,master,dbo,MSreplication_options,BASE TABLE


In [67]:
#Crio a nova tabela
conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()

crate_table_query = """
CREATE TABLE bitcoin_prices (
    id INT IDENTITY(1,1) PRIMARY KEY,
    symbol VARCHAR(10),
    timestamp DATETIME,
    [open] DECIMAL(18, 8),
    [high] DECIMAL(18, 8),
    [low] DECIMAL(18, 8),
    [close] DECIMAL(18, 8),
    volume DECIMAL(18, 8)
);
"""

try:
    cursor.execute(crate_table_query)
    print("Tabela crada com sucesso")
except Exception as e:
    print("Erro ao criar tabela", e)

cursor.close()
conn.close()

Tabela crada com sucesso


In [73]:
#Checo novamente se a tabela foi criada
df = checa_tabelas_existentes()

df[
    df['TABLE_NAME'] == 'bitcoin_prices' #Aplico um filtro para verificar se existe uma tabela com o nome qe eu criei
]

C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_20832\4279729226.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
5,master,dbo,bitcoin_prices,BASE TABLE
